# Our Goal

In the next 15 minutes, we're going to give you a rapid run through where you as a Data Owner, will prepare a dataset, deploy a domain node, upload that dataset to the domain node.
Finally, as a Data Scientist, you'll take this data and perform remote data science on it!

Note:
* We'll walk you through all the steps in much more detail in the next 3 lessons!

## Steps

Run the following commands in a Terminal:

### <b>1. Clone the PySyft repository, fetch and checkout the dev branch: </b>

git clone https://github.com/OpenMined/PySyft && cd PySyft
git fetch origin dev
git checkout dev

### <b>2. Launch <a href="https://www.docker.com/products/docker-desktop">Docker Desktop</a>, and ensure it has atleast 8GB of RAM available. </b>


### <b>3. Create a Virtual Environment with Python 3.9<b>

conda create -n lab python=3.9
conda activate lab

### <b>4. Install & Launch HAGrid</b>

cd packages/hagrid && pip install -e .
hagrid launch <node_name>

### <b>5. Install & Launch Syft </b>

cd packages/syft && pip install -e .

### <b>6. Login to the Domain!</b>


<i>Voila!</i> You now have a working domain node. To login, go to <b>localhost:"port_number"</b> in your browser (port_number is 8081 by default).

The default username and password are as follows:

- Username: <b> info@openmined.org </b>
- PW: <b> changethis </b>

### 7. Upload your Dataset to your Domain!

Open Python, and run the following:

In [ ]:
import numpy as np
data = np.random.choice([0, 1], size=(10))

import syft as sy
from syft.core.adp.entity import Entity

domain_node = sy.login(email="info@openmined.org", password="changethis", port=8081)
domain_node.load_dataset({"COVID Test Results": data})